In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [4]:
training = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [5]:
print(sum(np.sum(pd.isnull(training), axis = 0).tolist()))
# There are no NULL values in training dataset
print(sum(np.sum(pd.isnull(test), axis = 0).tolist()))
# There are no NULL values in test set

0
0


In [6]:
training = training.iloc[:, 1:]
test = test.iloc[:, 1:]

We want to recover from the data loss of the AGA frequency on test samples. Train a regressor which can predict the value of the AGA feature given the remaining ones. Compare different regression algorithms for this task. Since AGA features are missing in test samples, use only the training data for this step and make use of robust evaluation techniques to compare algorithms.

In [7]:
print("Kingdom values: ", np.unique(training["Kingdom"]))
print("DNA type values: ", np.unique(training["DNAtype"]))

Kingdom values:  ['arc' 'bct' 'inv' 'mam' 'phg' 'plm' 'pln' 'pri' 'rod' 'vrl' 'vrt']
DNA type values:  [ 0  1  2  3  4  5  6  7  9 11 12]


#### Function to one-hot encode categorical variables:

In [8]:
def onehot_feature(pd_data, column_name):
    # Retrieve the unique values (the categories) and an index for each sample
    # specifying the sample category (values[value_idx] reconstruct the original array)
    col_values = pd_data[column_name].to_numpy().astype('<U')
    values, value_idx = np.unique(col_values, return_inverse=True)
    n_values = values.size
    # Create a temporary identity matrix to convert value_idx into one-hot features
    onehots = np.eye(n_values) #when you use an array to index another array in NumPy, it selects rows from the indexed array based on the values in the index array. e[a] selects rows from the identity matrix e based on the values in array a
    value_onehot = onehots[value_idx]
    # Remove the categorical feature
    pd_data = pd_data.drop(column_name, axis=1)
    # Add the new featues
    for i in range(n_values):
        pd_data["{}_{}".format(column_name, values[i])] = value_onehot[:, i]
        
    return pd_data

#### One-hot encoding of both the 'Kingdom' and 'DNAtype' features (on both training and test datasets):

In [9]:
training_ohe = onehot_feature(training, 'Kingdom')
training_ohe = onehot_feature(training_ohe, 'DNAtype')

test_ohe = onehot_feature(test, 'Kingdom')
test_ohe = onehot_feature(test_ohe, 'DNAtype')

training_ohe.head()

,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,CUA,...,DNAtype_1,DNAtype_11,DNAtype_12,DNAtype_2,DNAtype_3,DNAtype_4,DNAtype_5,DNAtype_6,DNAtype_7,DNAtype_9
0,207343,6686,Bovine foamy virus,0.01541,0.00927,0.01720,0.01825,0.01436,0.02303,0.01555,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7629,10060,Anthocidaris crassispina,0.01213,0.02515,0.00268,0.01352,0.01372,0.01143,0.00775,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,320850,14031,Staphylococcus phage X2,0.02580,0.01618,0.03606,0.01354,0.00848,0.02067,0.01041,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1397,39839,Bacillus circulans,0.00128,0.02141,0.01225,0.01649,0.01330,0.01308,0.00399,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,50790,5112,mitochondrion Dascyllus trimaculatus,0.03169,0.01780,0.03893,0.00000,0.06240,0.01056,0.05262,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Correlation between AGA and each codon and then between each pair of codon

In [10]:
# First let's check if we can use Pearson correlation:

#for col in training.columns[5:]:
#    plt.hist(training.loc[:, col], bins = 10)
#    plt.show()
    
# We can't because they don't follow a gaussian distribution, so we use rank correlation methods like Spearman:

from scipy import stats
print("Codons that have a Spearman correlation >= 0.5 (in absolute terms) with AGA:")
for col in training.columns[5:]:
    if col != 'AGA':
        res = stats.spearmanr(training.loc[:, col], training.loc[:, 'AGA'])
        if abs(res.statistic) >= 0.5:
            print("{}: {} -- p-value = {}".format(col, res.statistic, res.pvalue))

# Now we look at the Spearman correlation between all codons to understand if there's some redundancy:

print("\nPairs of codons with a Spearman correlation >= 0.8 (in absolute terms) NOT considering AGA")
for codon1 in training.columns[5:]:
    for codon2 in training.columns[5:]:
        if codon1 != codon2 and codon1 != 'AGA' and codon2 != 'AGA':
            res = stats.spearmanr( training.loc[:, codon1]    ,  training.loc[:, codon2]    )
        if abs(res.statistic) >= 0.8:
            print("Spearman correlation between {} and {}: {} -- p-value = {}".format(codon1, codon2, res.statistic,
                                                                                     res.pvalue))

Codons that have a Spearman correlation >= 0.5 (in absolute terms) with AGA:
UUG: 0.517733529126989 -- p-value = 0.0
AUC: -0.6633343136412367 -- p-value = 0.0
GCC: -0.7147492089184697 -- p-value = 0.0
CCC: -0.5197634685183508 -- p-value = 0.0
GGC: -0.5852023884920887 -- p-value = 0.0
UCU: 0.5009233407342024 -- p-value = 0.0
AGU: 0.694694113172632 -- p-value = 0.0
UAU: 0.561316082441995 -- p-value = 0.0
AAU: 0.6609476920961472 -- p-value = 0.0
UGU: 0.669400904393223 -- p-value = 0.0
CAU: 0.5266827529444139 -- p-value = 0.0
CAC: -0.5115707636164963 -- p-value = 0.0
AAA: 0.5339363538522637 -- p-value = 0.0
AGG: 0.632327477358485 -- p-value = 0.0
GAU: 0.6161749105450396 -- p-value = 0.0
GAA: 0.5428476518370057 -- p-value = 0.0

Pairs of codons with a Spearman correlation >= 0.8 (in absolute terms) NOT considering AGA
Spearman correlation between UUA and AUU: 0.8367679640170536 -- p-value = 0.0
Spearman correlation between AUU and UUA: 0.8367679640170534 -- p-value = 0.0
Spearman correlatio

#### All functions needed later on:

In [11]:
from scipy.stats import t, f

def rss(y_true, y_pred):
    y_true = y_true.reshape(y_pred.shape)
    return np.sum((y_true - y_pred) ** 2)

def tss(y):
    return np.sum((y - y.mean()) ** 2)

def multiple_least_squares(X, y):
    model = LinearRegression(fit_intercept=True)
    model.fit(X, y)
    y_pred = model.predict(X)
    betas = [model.intercept_, *model.coef_]
    return betas, y_pred

def show_stats(X, y, betas, names, alpha=None):
    n_samples, n_features = X.shape
    deg = n_samples-n_features
    
    if X.shape[1] + 1 == betas.shape[0]:
        X = np.concatenate([np.ones([X.shape[0], 1]), X], axis=-1)
    
    pred = X.dot(betas).reshape(-1)
    betas = betas.reshape(-1)
    y = y.reshape(-1)
    RSE = ((y-pred)**2).sum()/(n_samples - n_features)
    
    se2_b = RSE*(np.linalg.inv(np.dot(X.T, X)).diagonal())
    se_b = np.sqrt(se2_b)
    t_stat_b = (betas - 0) / se_b

    p_values = np.array([2*t.sf(np.abs(t_stat), deg) for t_stat in t_stat_b])
    
    df = pd.DataFrame()
    df["Name"] = names
    df["Coefficients"] = betas
    df["Standard Errors"] = se_b
    df["t-stat"] = t_stat_b
    df["p-value"] = p_values
    if alpha:
        rejectH0 = p_values < alpha
        df["reject H0"] = rejectH0    
    
    RSS = np.sum((y - pred)**2)
    MSE = RSS/y.shape[0]
    return df
 
def Ftest_restricted(data, y, subset_features):
    X_complete = data.to_numpy()
    y = y.to_numpy()
    n = X_complete.shape[0]
    
    betas_complete, y_pred = multiple_least_squares(X_complete, y)    
    rss_complete = rss(y, y_pred)
    nf_complete = X_complete.shape[1]
    
    notS = data.columns.difference(subset_features)
    X_restr = data[notS].to_numpy()
    betas_restr, y_pred = multiple_least_squares(X_restr, y)

    rss_restr = rss(y, y_pred)
    nf_restr = X_restr.shape[1]

    q = nf_complete - nf_restr

    F_num = (rss_restr - rss_complete) / q
    F_den = rss_complete / (n - nf_complete - 1)
    F = F_num / F_den

    p_value = f.sf(F, q, n - nf_complete - 1)
    return p_value, F

#### Multivariate linear regression models to understand which codon are useless for predicting AGA frequency
#### Features considered: Codons

In [12]:
from sklearn.linear_model import LinearRegression

X1 = training.iloc[:, 5:].drop("AGA", axis = 1)
y = training.loc[:, "AGA"]

X1_all_features = [col_name for col_name in X1.columns.tolist()]

linear_model1 = LinearRegression(fit_intercept=True)
linear_model1 = linear_model1.fit(X1, y)

betas = np.array([linear_model1.intercept_, *linear_model1.coef_]).reshape(-1, 1) 
final_stats = show_stats(X1.to_numpy(), y.to_numpy(), betas, ['Intercept', *X1_all_features], alpha=0.001)
print(final_stats)
print("\nNOT SIGNIFICANT CODONS:\n ", final_stats[final_stats.loc[:, 'reject H0']==False])

         Name  Coefficients  Standard Errors        t-stat  p-value  reject H0
0   Intercept      0.999938         0.000082  12248.651521      0.0       True
1         UUU     -0.999900         0.000088 -11419.511160      0.0       True
2         UUC     -0.999959         0.000101  -9905.781604      0.0       True
3         UUA     -0.999960         0.000096 -10412.822448      0.0       True
4         UUG     -0.999897         0.000103  -9710.379087      0.0       True
..        ...           ...              ...           ...      ...        ...
59        GAA     -0.999955         0.000103  -9738.646816      0.0       True
60        GAG     -0.999950         0.000100 -10007.550693      0.0       True
61        UAA     -0.999893         0.000088 -11412.835860      0.0       True
62        UAG     -0.999394         0.000435  -2295.279831      0.0       True
63        UGA     -1.000029         0.000133  -7535.083484      0.0       True

[64 rows x 6 columns]

NOT SIGNIFICANT CODONS:
  Em

#### Multivariate linear regression models to understand which codon are useless for predicting AGA frequency
#### Features considered: Codons, Kingdom and DNAtype

#### Se consideriamo tutti i livelli della dummy variable "DNAtype" la funzione dà NaN per gli SE dei coefficienti --> può aver senso rimuovere i livelli della variabile che hanno troppe poche osservazioni (1 sola) dato che questo porta sicuramente ad una stima non precisa del coefficiente

In [13]:
X2 = training_ohe.iloc[:, 3:].drop(["AGA", "DNAtype_5", "DNAtype_11"], axis = 1)
y = training_ohe.loc[:, "AGA"]

X2_all_features = X2.columns.tolist()

linear_model2 = LinearRegression(fit_intercept=True)
linear_model2 = linear_model2.fit(X2, y)

betas = np.array([linear_model2.intercept_, *linear_model2.coef_]).reshape(-1, 1) 
final_stats = show_stats(X2.to_numpy(), y.to_numpy(), betas, ['Intercept', *X2_all_features], alpha=0.001)


print(final_stats)
print("\nNOT SIGNIFICANT FEATURES: \n", final_stats[final_stats.loc[:, 'reject H0']==False])

         Name  Coefficients  Standard Errors        t-stat   p-value  \
0   Intercept      0.999952        75.650494      0.013218  0.989454   
1         UUU     -0.999915         0.000091 -11009.097462  0.000000   
2         UUC     -0.999984         0.000105  -9564.175374  0.000000   
3         UUA     -0.999974         0.000101  -9937.690367  0.000000   
4         UUG     -0.999913         0.000105  -9495.334790  0.000000   
..        ...           ...              ...           ...       ...   
79  DNAtype_3     -0.000033         0.000038     -0.868047  0.385389   
80  DNAtype_4     -0.000008         0.000028     -0.286691  0.774355   
81  DNAtype_6      0.000034         0.000047      0.712614  0.476100   
82  DNAtype_7      0.000040         0.000047      0.855893  0.392077   
83  DNAtype_9     -0.000046         0.000039     -1.187702  0.234978   

    reject H0  
0       False  
1        True  
2        True  
3        True  
4        True  
..        ...  
79      False  
80     

#### Performance of the model with only the codons


In [14]:
from sklearn.metrics import r2_score, mean_squared_error

y_predict = linear_model1.predict(X1)
print("Train R2 score ", r2_score(y, y_predict))
print("Train MSE score ", mean_squared_error(y, y_predict))

Train R2 score  0.9999803188956894
Train MSE score  1.4547727981251517e-09


#### Check that 'DNAtype' and 'Kingdom' features are actually useless to predict AGA frequency
By looking at the p-value, we can exclude those features.

In [15]:
features_to_exclude = X2.loc[:, "Kingdom_arc":].columns.tolist()

p_value, F = Ftest_restricted(X2, y, features_to_exclude)
print("p-value =", p_value)
print("F-stat =", F)
if p_value < 0.001:
    print("Reject H0: There is evidence to say that at least one of the S features is useful")
else:
    print("Do not Reject H0: There is NO evidence to say that at least one of the S features is useful")


p-value = 0.5937615870539219
F-stat = 0.8951735779740977
Do not Reject H0: There is NO evidence to say that at least one of the S features is useful


## METTERE PLOT SUI RESIDUI PER CAPIRE SE USARE POLINOMIALI ETC


### Ridge vs Lasso
#### In theory, Lasso performs better when we expect only few features to be significant. In this case we expect a lot of features to be useful (almost all the codons) so probably Ridge is better (try both but with this premise)
#### Don't forget about Elastic Net

#### Ridge, with CV

In [16]:
from sklearn.linear_model import RidgeCV
X = training_ohe.iloc[:, 3:].drop("AGA", axis = 1)
y = training_ohe.loc[:, "AGA"]

ridge_linear_model = RidgeCV(alphas=[1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4,
                                    1e-3, 1e-2, 1e-1, 1], fit_intercept = True).fit(X, y)
ridge_linear_model.score(X, y)
ridge_linear_model.alpha_ # Regolarizza poco

1e-08

#### Lasso, with CV

In [17]:
from sklearn.linear_model import LassoCV
X = training_ohe.iloc[:, 3:].drop("AGA", axis = 1)
y = training_ohe.loc[:, "AGA"]

lasso_linear_model = LassoCV(cv=10, random_state=0).fit(X, y)
lasso_linear_model.score(X, y)
for idx,coef in enumerate(lasso_linear_model.coef_):
    if abs(coef) != 0:
        pass
        #print("Coefficient of {}: {}".format(X.columns[idx], coef))
    else:
        print("Coefficient {} was removed".format(X.columns[idx]))


Coefficient UUU was removed
Coefficient UUG was removed
Coefficient CUC was removed
Coefficient AUG was removed
Coefficient GUU was removed
Coefficient GUC was removed
Coefficient GUA was removed
Coefficient GCU was removed
Coefficient CCU was removed
Coefficient CCA was removed
Coefficient CCG was removed
Coefficient GGG was removed
Coefficient UCC was removed
Coefficient UCA was removed
Coefficient UCG was removed
Coefficient AGU was removed
Coefficient ACU was removed
Coefficient ACC was removed
Coefficient UAU was removed
Coefficient CAG was removed
Coefficient UGU was removed
Coefficient UGC was removed
Coefficient CAU was removed
Coefficient CAC was removed
Coefficient AAG was removed
Coefficient CGA was removed
Coefficient CGG was removed
Coefficient GAU was removed
Coefficient GAG was removed
Coefficient UAA was removed
Coefficient UAG was removed
Coefficient UGA was removed
Coefficient Kingdom_bct was removed
Coefficient Kingdom_phg was removed
Coefficient Kingdom_plm was remo